# RAG Demo with OpenRouter (GPT-3.5) and HuggingFace Embeddings

This notebook demonstrates a Retrieval Augmented Generation (RAG) system using:
- OpenRouter with GPT-3.5 Turbo for the LLM
- HuggingFace sentence-transformers for embeddings

In [1]:
# Import necessary libraries
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

# Verify API key is loaded
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in .env file")
else:
    print("OpenRouter API key loaded successfully")

OpenRouter API key loaded successfully


In [3]:
# Configure OpenRouter base URL and headers
openrouter_base_url = "https://openrouter.ai/api/v1"
openrouter_headers = {
    "HTTP-Referer": "https://localhost:8888",  # Required by OpenRouter
    "X-Title": "RAG Demo"  # Optional, for tracking in OpenRouter dashboard
}

In [4]:
# Create LLM instance using OpenRouter with GPT-3.5
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_base=openrouter_base_url,
    openai_api_key=OPENROUTER_API_KEY,
    temperature=0.4,
    max_tokens=500,
    default_headers=openrouter_headers
)

In [5]:
# Test the LLM to make sure it's working
response = llm.invoke("Hello, are you using OpenRouter with GPT-3.5?")
print(response.content)

Hello! Yes, I am using OpenAI's GPT-3.5 model to assist you with any questions or tasks you have. How can I help you today?


In [6]:
# Load URLs
urls = [
    'https://www.victoriaonmove.com.au/local-removalists.html',
    'https://victoriaonmove.com.au/index.html',
    'https://victoriaonmove.com.au/contact.html'
]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [7]:
# View the loaded data
data

[Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\

In [8]:
# Split data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)
print("Total number of documents: ", len(docs))

Total number of documents:  11


In [9]:
# View a sample document
docs[0]

Document(metadata={'source': 'https://www.victoriaonmove.com.au/local-removalists.html'}, page_content='Loading...\n\nLOCAL REMOVALS\n\nYour trusted partner in seamless moving and packing solutions!\n\nGoogle logo G\n\nGoogle Rating\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\nGold star icon\n\n5 stars, 111 reviews\n\n\n\nRequst A call for You:\n\nLocal removal services via "Victoria on move"\n\nVictoria on Move is your trusted local moving company in Melbourne, specializing in seamless relocation services. As experienced furniture movers and relocation experts, we provide top-notch packing and moving services tailored to your needs. Whether you\'re moving across town or relocating interstate, our professional movers ensure a stress-free experience. Count on Victoria on Move for reliable removal services, making us the preferred choice among local movers in Melbourne. Discover why we\'re recognized for our commitment to quality and customer satisfaction.\n

In [10]:
# Set up HuggingFace embeddings (local model)
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  # Small, fast model that works well
    model_kwargs={'device': 'cpu'}
)

C:\Windows\Temp\ipykernel_2772\1729983770.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
d:\LLM_OPS\Rag\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Create vectorstore with the embeddings
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [12]:
# Create retriever from vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [13]:
# Test retriever with a question
retrieved_docs = retriever.invoke("What kind of services they provide?")
print(f"Retrieved {len(retrieved_docs)} documents")

Retrieved 3 documents


In [14]:
# View the first retrieved document
print(retrieved_docs[0].page_content)

Unlike some other moving companies, we value the trust our customers place in us. Our customer reviews on Google are genuine, and we never manipulate them. We trust in our quality and experience and believe in our destiny to provide top-notch moving and packing services to our clients.

Contact Us

Our Clients Say!

Discover firsthand experiences from our valued clients through their heartfelt testimonials. From seamless moves to exceptional service, our customers share how we've made their relocation journey stress-free and rewarding. Explore their stories and see why they trust us with their moves time and again.





















Get In Touch

Wollert Victoria

0404922328

victoriaonmove07@gmail.com

Quick Links

About Us Contact Us Our Services Terms & Condition

Photo Gallery













Check us out on Google!

reviews

© Victoria On Move 2024, All Right Reserved. Designed By HTML Codex

Home


In [15]:
# Create a prompt template for the RAG chain
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [16]:
# Create the RAG chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [17]:
# Test the RAG chain with a question
response = rag_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])

Victoria on Move provides a range of services including apartment moving, villa moving, household moving, office moving, furniture moving, and packing/unpacking services. They offer efficient and careful relocation solutions tailored to different needs, ensuring a stress-free moving experience for their clients. Additionally, they specialize in customized moving plans to cater to specific requirements.


In [18]:
# Try another question
response = rag_chain.invoke({"input": "What are their moving truck options and prices?"})
print(response["answer"])

Victoria On Move offers four moving truck options with prices starting from $110 per hour for a Small Truck with 2 movers, suitable for a student apartment or small office. The Medium Truck with 2 movers starts from $130 per hour and can handle 2-3 bedrooms, while the Large Truck with 2 movers starts from $140 per hour and can manage 3-4 bedrooms. The X-Large Truck with 2 movers starts from $150 per hour and can handle 3-4 bedrooms with double lounge and outdoor items.


In [19]:
# Try a question about their contact information
response = rag_chain.invoke({"input": "How can I contact Victoria on Move?"})
print(response["answer"])

You can contact Victoria on Move by phone at 0404922328 or via email at victoriaonmove07@gmail.com. They are located in Wollert, Victoria, and offer local removalist services in Melbourne. Additionally, you can visit their website for more information and to request a call back for their local removal services.
